<a href="https://colab.research.google.com/github/boyoung21/ESAA/blob/main/0919_%EC%88%98%EC%83%81%EC%9E%91%EB%A6%AC%EB%B7%B0_%EB%8C%80%EA%B5%AC_%EA%B5%90%ED%86%B5%EC%82%AC%EA%B3%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 대구 교통사고 피해 예측 경진대회

https://dacon.io/competitions/official/236193/overview/description

대회 목적: 시공간 정보로부터 사고위험도(ECLO) 예측 AI 모델 개발

Dataset Info.

1. train.csv [파일]
- ID : 대구에서 발생한 교통사고의 고유 ID
> 2019년부터 2021년까지의 교통사고 데이터로 구성. 해당 사고가 발생한 당시의 시공간 정보와 사고 관련 정보 포함
- ECLO : 인명피해 심각도

2. test.csv [파일]
- ID : 대구에서 발생한 교통사고의 고유 ID
> 2022년도의 교통사고 데이터로 구성. 추론 시점에서 획득할 수 있는 정보로 구성

3. sample_submission.csv [파일] - 제출 양식
- ID : 추론 샘플의 고유 ID
- ECLO : 예측한 인명피해 심각도


- 대구 빅데이터 마트 데이터 [폴더]
> 대구 빅데이터활용센터에서 구축한 빅데이터 마트 데이터 중 제공 가능한 일부 데이터셋. 전체 빅데이터 마트 데이터셋을 활용하기 위해서는 대구 빅데이터활용센터를 직접 방문하여 사내망 사용

- countrywide_accident.csv [파일]
> 대구를 제외한 전국에서 발생한 교통사고 데이터. 2019년부터 2021년까지의 교통사고 데이터로 구성

- 대구 보안등 정보.csv [파일]
> 대구에 존재하는 보안등 관련 정보

- 대구 어린이 보호 구역 정보.csv [파일]
> 대구에 존재하는 어린이 보호 구역 관련 정보

- 대구 주차장 정보.csv [파일]
> 대구에 존재하는 주차장 관련 정보

- 대구 CCTV 정보.csv [파일]
> 대구에 존재하는 CCTV 관련 정보
1. 도로노선방향
> 01 : 상행
02 : 하행
03 : 양방향
- 단속구분
> 01 : 속도
02 : 신호
03 : 통행위반
04 : 불법주정차
99 : 기타
- 단속구간위치구분
> 01 : 시점
02 : 종점
- 보호구역구분
> 01 : 노인보호구역
02 : 어린이보호구역
99 : 기타



# EDA

- 외부 데이터 적극적 활용: 데이터의 시공간 정보로부터 사고위험도를 유추하기 때문에, 시공간 정보가 많을수록 예측이 잘될 것이라고 판단함. 따라서 공공데이터 포털, 외부 데이터를 추가적으로 활용

- 시공간적으로 어떤 부분이 target에 영향을 미치는지에 주목

> 시간별 사고위험도를 파악함. 0시~5시에는 타깃값이 높으나 이 외의 시간대에는 타깃값이 낮은 주기성을 확인 (시계열성)

> 평일에 비해 주말에 타깃값이 더 높은 주기성 확인

> 특정 구에서 다른 구에 비해 더 높은 사고위험도 보이는 것 확인

> 구보다 더 세밀한 동별 타깃값 비교한 결과, 고속도로나 산간지역에 위치한 동일수록 타깃값이 높게 나타나는 편차 확인

> CCTV 제한 속도가 높아질수록 타깃값도 높아지는 것 확인

-> 데이터를 바탕으로 실제 지형적 특징을 결합시켜 분석을 도출해 낸 점이 인상 깊음. 이전에 집값 예측 데이터를 다룰 때 지리적 특성을 어떻게 사용해야 할지 막막해서 제대로 활용하지 못했는데, 참고가 됨.

# 데이터 전처리

- 시간 feature를 시간대별로 그룹핑
- 평일, 토요일, 일요일을 오름차순으로 라벨인코딩
- 시계열성을 다루기 위해 시간 및 날짜 피처들에 sin / cos 활용
- 파생변수 생성

> 구별 ECLO값의 평균이 5보다 큰 구를 뽑아 그 구에 속하는 지 여부에 대한 파생변수 생성

> 동별 ECLO값의 평균이 5보다 큰 동을 뽑아 그 동에 속하는 지 여부에 대한 파생변수 생성

> 동별 사망자,중상자,경상자 및 부상자 수를 sum하는 파생변수 생성

# 모델링

- 범주형 변수로 구성된 공간정보를 활용하기 위해 CatBoost 사용
- 날짜 및 시간정보를 시계열 정보로 활용하기 위해 XGBoost 사용
- Overfitting을 방지하기 위해 별도의 파라미터 튜닝은 하지 않음
- 여러 개의 교차 검증 활용(Multi-StratifiedKFold cross validation)